<a href="https://colab.research.google.com/github/1013115/KTB_Project/blob/main/%EB%82%A0%EC%94%A8%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%95%EB%A6%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## api key 호출

In [1]:
from google.colab import userdata
data_servicekey = userdata.get('DATA_API_KEY')
weather_servicekey = userdata.get('WEATHER_API_KEY')

## 주소 변환

참고 : https://gist.github.com/fronteer-kr/14d7f779d52a21ac2f16

In [5]:
import math
NX = 149            ## X축 격자점 수
NY = 253            ## Y축 격자점 수

Re = 6371.00877     ##  지도반경
grid = 5.0          ##  격자간격 (km)
slat1 = 30.0        ##  표준위도 1
slat2 = 60.0        ##  표준위도 2
olon = 126.0        ##  기준점 경도
olat = 38.0         ##  기준점 위도
xo = 210 / grid     ##  기준점 X좌표
yo = 675 / grid     ##  기준점 Y좌표
first = 0

if first == 0 :
    PI = math.asin(1.0) * 2.0
    DEGRAD = PI/ 180.0
    RADDEG = 180.0 / PI


    re = Re / grid
    slat1 = slat1 * DEGRAD
    slat2 = slat2 * DEGRAD
    olon = olon * DEGRAD
    olat = olat * DEGRAD

    sn = math.tan(PI * 0.25 + slat2 * 0.5) / math.tan(PI * 0.25 + slat1 * 0.5)
    sn = math.log(math.cos(slat1) / math.cos(slat2)) / math.log(sn)
    sf = math.tan(PI * 0.25 + slat1 * 0.5)
    sf = math.pow(sf, sn) * math.cos(slat1) / sn
    ro = math.tan(PI * 0.25 + olat * 0.5)
    ro = re * sf / math.pow(ro, sn)
    first = 1

def mapToGrid(lat, lon, code = 0 ):
    ra = math.tan(PI * 0.25 + lat * DEGRAD * 0.5)
    ra = re * sf / pow(ra, sn)
    theta = lon * DEGRAD - olon
    if theta > PI :
        theta -= 2.0 * PI
    if theta < -PI :
        theta += 2.0 * PI
    theta *= sn
    x = (ra * math.sin(theta)) + xo
    y = (ro - ra * math.cos(theta)) + yo
    x = int(x + 1.5)
    y = int(y + 1.5)
    return x, y

def gridToMap(x, y, code = 1):
    x = x - 1
    y = y - 1
    xn = x - xo
    yn = ro - y + yo
    ra = math.sqrt(xn * xn + yn * yn)
    if sn < 0.0 :
        ra = -ra
    alat = math.pow((re * sf / ra), (1.0 / sn))
    alat = 2.0 * math.atan(alat) - PI * 0.5
    if math.fabs(xn) <= 0.0 :
        theta = 0.0
    else :
        if math.fabs(yn) <= 0.0 :
            theta = PI * 0.5
            if xn < 0.0 :
                theta = -theta
        else :
            theta = math.atan2(xn, yn)
    alon = theta / sn + olon
    lat = alat * RADDEG
    lon = alon * RADDEG

    return lat, lon

print(mapToGrid(37.400383, 127.106706))

print(gridToMap(62, 123))


(62, 123)
(37.39424102222431, 127.10252748850417)


### Geopy의 Nominatim 모듈을 사용한 주소변환

참고 : https://parkgihyeon.github.io/project/geocoding-api/

In [3]:
from geopy.geocoders import Nominatim

def geocoding_reverse(lat_lng_str):
    geolocoder = Nominatim(user_agent='South Korea', timeout=None)
    location = geolocoder.reverse(lat_lng_str)

    # address 객체에서 원하는 정보를 추출
    address_details = location.raw.get('address', {})
    city = address_details.get('city', '')  # 시 정보
    district = address_details.get('suburb', '') or address_details.get('town', '') or address_details.get('village', '') or address_details.get('hamlet', '')  # 동/읍/면 정보

    # 원하는 형식으로 반환
    return f"{city} {district}"

address = geocoding_reverse('36.5760732781656, 128.15935928504484')
print(address)


상주시 교촌리


In [16]:
# 공공데이터 포털 url
# 초단기 예보
data_url = f"https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst?serviceKey={data_servicekey}&numOfRows=10&pageNo=1&base_date=20241101&base_time=0630&nx=62&ny=123&dataType=JSON"
# 단기 예보
data_url = f"https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey={data_servicekey}&numOfRows=10&pageNo=1&base_date=20241101&base_time=0500&nx=62&ny=123&dataType=JSON"

# 기상청
# 초단기 예보
weather_url = f"https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getUltraSrtFcst?authKey={weather_servicekey}&numOfRows=500&pageNo=1&base_date=20241101&base_time=0630&nx=62&ny=123&dataType=JSON"
# 단기 예보
#weather_url = f"https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst?authKey={weather_servicekey}&numOfRows=500&pageNo=1&base_date=20241101&base_time=0500&nx=62&ny=123&dataType=JSON"


In [32]:
import requests
import json
response = requests.get(weather_url, verify=False)
res = json.loads(response.text)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apihub.kma.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## 코드 변환

참고 : https://velog.io/@choonsik_mom/python-OpenAPI%EA%B8%B0%EC%83%81%EC%B2%AD-API%EB%A1%9C-%EB%82%A0%EC%94%A8-%EC%A0%95%EB%B3%B4-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0


In [11]:
information = dict()

for item in res['response']['body']['items']['item']:
    # 날짜와 시간 정보를 가져옵니다.
    fcstDate = item['fcstDate']  # 날짜 정보
    fcstTime = item['fcstTime']  # 시간 정보
    cate = item['category']      # 카테고리 정보
    fcstValue = item['fcstValue']  # 예보 값

    # 날짜와 시간을 결합하여 고유한 키를 생성합니다.
    fcstDateTime = f"{fcstDate}_{fcstTime}"

    # fcstDateTime이 information에 없으면 새 딕셔너리를 생성합니다.
    if fcstDateTime not in information:
        information[fcstDateTime] = dict()

    # 예보 카테고리와 값을 추가합니다.
    information[fcstDateTime][cate] = fcstValue

# 결과 출력
print(information)


{'20241101_0600': {'TMP': '11', 'UUU': '-0.8', 'VVV': '0.1', 'VEC': '98', 'WSD': '0.9', 'SKY': '3', 'PTY': '0', 'POP': '20', 'WAV': '0', 'PCP': '강수없음', 'REH': '85', 'SNO': '적설없음'}, '20241101_0700': {'TMP': '11', 'UUU': '-0.4', 'VVV': '-0.3', 'VEC': '58', 'WSD': '0.5', 'SKY': '4', 'PTY': '0', 'POP': '30', 'WAV': '0', 'PCP': '강수없음', 'REH': '85', 'SNO': '적설없음'}, '20241101_0800': {'TMP': '11', 'UUU': '0', 'VVV': '-0.4', 'VEC': '14', 'WSD': '0.4', 'SKY': '4', 'PTY': '0', 'POP': '30', 'WAV': '0', 'PCP': '강수없음', 'REH': '80', 'SNO': '적설없음'}, '20241101_0900': {'TMP': '13', 'UUU': '-0.3', 'VVV': '-0.2', 'VEC': '67', 'WSD': '0.4', 'SKY': '4', 'PTY': '0', 'POP': '30', 'WAV': '0', 'PCP': '강수없음', 'REH': '75', 'SNO': '적설없음'}, '20241101_1000': {'TMP': '14', 'UUU': '-0.6', 'VVV': '0.2', 'VEC': '106', 'WSD': '0.7', 'SKY': '4', 'PTY': '0', 'POP': '30', 'WAV': '0', 'PCP': '강수없음', 'REH': '65', 'SNO': '적설없음'}, '20241101_1100': {'TMP': '16', 'UUU': '-1.1', 'VVV': '0.3', 'VEC': '106', 'WSD': '1.2', 'SKY': '4'

In [12]:
deg_code = {0 : 'N', 360 : 'N', 180 : 'S', 270 : 'W', 90 : 'E', 22.5 :'NNE',
           45 : 'NE', 67.5 : 'ENE', 112.5 : 'ESE', 135 : 'SE', 157.5 : 'SSE',
           202.5 : 'SSW', 225 : 'SW', 247.5 : 'WSW', 292.5 : 'WNW', 315 : 'NW',
           337.5 : 'NNW'}

def deg_to_dir(deg) :
    close_dir = ''
    min_abs = 360
    if deg not in deg_code.keys() :
        for key in deg_code.keys() :
            if abs(key - deg) < min_abs :
                min_abs = abs(key - deg)
                close_dir = deg_code[key]
    else :
        close_dir = deg_code[deg]
    return close_dir
deg_to_dir(0)

'N'

### 단기예보

In [31]:
pyt_code = {0 : '강수 없음', 1 : '비', 2 : '비/눈', 3 : '눈', 5 : '빗방울', 6 : '진눈깨비', 7 : '눈날림'}
sky_code = {1 : '맑음', 3 : '구름많음', 4 : '흐림'}
address = geocoding_reverse(gridToMap(62, 123))

for key, val in zip(information.keys(), information.values()) :
#     print(key, val)
    # val['LGT'] -- 낙뢰
    template = f"""{key[:4]}년 {key[4:6]}월 {key[6:8]}일 {key[9:11]}시 {key[11:]}분 {address} 날씨: """


    # 맑음(1), 구름많음(3), 흐림(4)
    try:
      if val['SKY'] :
          sky_temp = sky_code[int(val['SKY'])]
  #         print("하늘 :", sky_temp)
          template += sky_temp + " | "
    except KeyError:
        # tmn 키가 없는 경우 아무 작업도 하지 않고 넘어감
        pass
    # (초단기) 없음(0), 비(1), 비/눈(2), 눈(3), 빗방울(5), 빗방울눈날림(6), 눈날림(7)
    try:
      if val['PTY'] :
          pty_temp = pyt_code[int(val['PTY'])]
  #         print("강수 여부 :",pty_temp)
          template += pty_temp + " | "
          # 강수 있는 경우
          if val['POP'] :
              # POP 강수확률
              pop_temp = val['POP']
  #             print("강수량(1시간당) :",rn1_temp)
              template += f"강수확률 {pop_temp}% | "
          if val['PCP'] != '강수없음' :
              # PCP 1시간 강수량
              pcp_temp = val['PCP']
  #             print("강수량(1시간당) :",rn1_temp)
              template += f"시간당 {pcp_temp}mm | "
    except KeyError:
        # tmn 키가 없는 경우 아무 작업도 하지 않고 넘어감
        pass




    # 기온
    if val['TMP'] :
        t1h_temp = float(val['TMP'])
#         print(f"기온 : {t1h_temp}℃")
        template += f" 기온 {t1h_temp}℃"+ " | "
    # 습도
    try:
      if val['REH'] :
          reh_temp = float(val['REH'])
  #         print(f"습도 : {reh_temp}%")
          template += f"습도 {reh_temp}%"+ "| "
    except KeyError:
        # tmn 키가 없는 경우 아무 작업도 하지 않고 넘어감
        pass


    # 풍향/ 풍속
    if val['VEC'] and val['WSD']:
        vec_temp = deg_to_dir(float(val['VEC']))
        wsd_temp = val['WSD']
#         print(f"풍속 :{vec_temp} 방향 {wsd_temp}m/s")

    template += f"풍속 {vec_temp} 방향 {wsd_temp}m/s"+ " | "

    try:
        tmn_temp = val['TMN']
        template += f"최저 기온 {tmn_temp}℃ | "
    except KeyError:
        # tmn 키가 없는 경우 아무 작업도 하지 않고 넘어감
        pass
    try:
        tmx_temp = val['TMX']
        template += f"최고 기온 {tmx_temp}℃ | "
    except KeyError:
        # tmn 키가 없는 경우 아무 작업도 하지 않고 넘어감
        pass



    print(template)

2024년 11월 01일 06시 00분 성남시 판교 날씨: 구름많음 | 강수 없음 | 강수확률 20% |  기온 11.0℃ | 습도 85.0%| 풍속 E 방향 0.9m/s | 
2024년 11월 01일 07시 00분 성남시 판교 날씨: 흐림 | 강수 없음 | 강수확률 30% |  기온 11.0℃ | 습도 85.0%| 풍속 ENE 방향 0.5m/s | 
2024년 11월 01일 08시 00분 성남시 판교 날씨: 흐림 | 강수 없음 | 강수확률 30% |  기온 11.0℃ | 습도 80.0%| 풍속 NNE 방향 0.4m/s | 
2024년 11월 01일 09시 00분 성남시 판교 날씨: 흐림 | 강수 없음 | 강수확률 30% |  기온 13.0℃ | 습도 75.0%| 풍속 ENE 방향 0.4m/s | 
2024년 11월 01일 10시 00분 성남시 판교 날씨: 흐림 | 강수 없음 | 강수확률 30% |  기온 14.0℃ | 습도 65.0%| 풍속 ESE 방향 0.7m/s | 
2024년 11월 01일 11시 00분 성남시 판교 날씨: 흐림 | 강수 없음 | 강수확률 30% |  기온 16.0℃ | 습도 60.0%| 풍속 ESE 방향 1.2m/s | 
2024년 11월 01일 12시 00분 성남시 판교 날씨: 맑음 | 강수 없음 | 강수확률 0% |  기온 16.0℃ | 습도 60.0%| 풍속 ENE 방향 0.7m/s | 
2024년 11월 01일 13시 00분 성남시 판교 날씨: 맑음 | 강수 없음 | 강수확률 0% |  기온 17.0℃ | 습도 55.0%| 풍속 NE 방향 1m/s | 
2024년 11월 01일 14시 00분 성남시 판교 날씨: 맑음 | 강수 없음 | 강수확률 0% |  기온 18.0℃ | 습도 55.0%| 풍속 E 방향 1.3m/s | 
2024년 11월 01일 15시 00분 성남시 판교 날씨: 맑음 | 강수 없음 | 강수확률 0% |  기온 18.0℃ | 습도 55.0%| 풍속 NNE 방향 1.7m/s | 최고 기온 19.0℃ | 
2024년

### 초단기 예보

In [28]:
pyt_code = {0 : '강수 없음', 1 : '비', 2 : '비/눈', 3 : '눈', 5 : '빗방울', 6 : '진눈깨비', 7 : '눈날림'}
sky_code = {1 : '맑음', 3 : '구름많음', 4 : '흐림'}
address = geocoding_reverse(gridToMap(62, 123))

for key, val in zip(information.keys(), information.values()) :
#     print(key, val)
    # val['LGT'] -- 낙뢰
    template = f"""{key[:4]}년 {key[4:6]}월 {key[6:8]}일 {key[9:11]}시 {key[11:]}분 {address} 날씨: """


    # 맑음(1), 구름많음(3), 흐림(4)
    try:
      if val['SKY'] :
          sky_temp = sky_code[int(val['SKY'])]
  #         print("하늘 :", sky_temp)
          template += sky_temp + " | "
    except KeyError:
        pass

    # (초단기) 없음(0), 비(1), 비/눈(2), 눈(3), 빗방울(5), 빗방울눈날림(6), 눈날림(7)
    try:
      if val['PTY'] :
          pty_temp = pyt_code[int(val['PTY'])]
  #         print("강수 여부 :",pty_temp)
          template += pty_temp+" | "
          # 강수 있는 경우
          try:
            if val['RN1'] != '강수없음' :
                # RN1 1시간 강수량
                rn1_temp = val['RN1']
    #             print("강수량(1시간당) :",rn1_temp)
                template += f"시간당 {rn1_temp}mm | "
          except KeyError:
              #  키가 없는 경우 아무 작업도 하지 않고 넘어감
              pass
    except KeyError:
        pass

    # 기온
    try:
      if val['T1H'] :
          t1h_temp = float(val['T1H'])
  #         print(f"기온 : {t1h_temp}℃")
          template += f"기온 {t1h_temp}℃ | "
    except KeyError:
      pass
    # 습도
    try:
      if val['REH'] :
          reh_temp = float(val['REH'])
  #         print(f"습도 : {reh_temp}%")
          template += f"습도 {reh_temp}% | "
    except KeyError:
      pass


    # 풍향/ 풍속
    if val['VEC'] and val['WSD']:
        vec_temp = deg_to_dir(float(val['VEC']))
        wsd_temp = val['WSD']
#         print(f"풍속 :{vec_temp} 방향 {wsd_temp}m/s")

    template += f"풍속 {vec_temp} 방향 {wsd_temp}m/s | "
    print(template)

2024년 11월 01일 06시 00분 성남시 판교 날씨: 구름많음 | 강수 없음 | 습도 85.0% | 풍속 E 방향 0.9m/s | 
2024년 11월 01일 07시 00분 성남시 판교 날씨: 흐림 | 강수 없음 | 습도 85.0% | 풍속 ENE 방향 0.5m/s | 
2024년 11월 01일 08시 00분 성남시 판교 날씨: 흐림 | 강수 없음 | 습도 80.0% | 풍속 NNE 방향 0.4m/s | 
2024년 11월 01일 09시 00분 성남시 판교 날씨: 흐림 | 강수 없음 | 습도 75.0% | 풍속 ENE 방향 0.4m/s | 
2024년 11월 01일 10시 00분 성남시 판교 날씨: 흐림 | 강수 없음 | 습도 65.0% | 풍속 ESE 방향 0.7m/s | 
2024년 11월 01일 11시 00분 성남시 판교 날씨: 흐림 | 강수 없음 | 습도 60.0% | 풍속 ESE 방향 1.2m/s | 
2024년 11월 01일 12시 00분 성남시 판교 날씨: 맑음 | 강수 없음 | 습도 60.0% | 풍속 ENE 방향 0.7m/s | 
2024년 11월 01일 13시 00분 성남시 판교 날씨: 맑음 | 강수 없음 | 습도 55.0% | 풍속 NE 방향 1m/s | 
2024년 11월 01일 14시 00분 성남시 판교 날씨: 맑음 | 강수 없음 | 습도 55.0% | 풍속 E 방향 1.3m/s | 
2024년 11월 01일 15시 00분 성남시 판교 날씨: 맑음 | 강수 없음 | 습도 55.0% | 풍속 NNE 방향 1.7m/s | 
2024년 11월 01일 16시 00분 성남시 판교 날씨: 맑음 | 강수 없음 | 습도 60.0% | 풍속 N 방향 0.8m/s | 
2024년 11월 01일 17시 00분 성남시 판교 날씨: 맑음 | 강수 없음 | 습도 65.0% | 풍속 NNE 방향 1.1m/s | 
2024년 11월 01일 18시 00분 성남시 판교 날씨: 맑음 | 강수 없음 | 습도 70.0% | 풍속 NNE 방향 1m/s | 
2024년 11